In [1]:
import pandas as pd

## First Part

In [2]:
d=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [3]:
len(d)

3

Scraped data has 3 table. Check the web page and get the collect table from data.

In [4]:
df=d[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Postal Code   180 non-null    object
 1   Borough       180 non-null    object
 2   Neighborhood  180 non-null    object
dtypes: object(3)
memory usage: 4.3+ KB


In [6]:
df_new=df[df['Borough']!='Not assigned']
df_new.head()


,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
df_new.describe()

,Postal Code,Borough,Neighborhood
count,103,103,103
unique,103,10,99
top,M3M,North York,Downsview
freq,1,24,4


In [8]:
#df_new['Neighborhood'].describe()

In [9]:
df_new['Postal Code'].describe()

count     103
unique    103
top       M3M
freq        1
Name: Postal Code, dtype: object

All postal code are unique. No need to merge anything

In [10]:
d=df_new[df_new['Neighborhood']=='Not assigned']
d

,Postal Code,Borough,Neighborhood


No neighborhood which is not assigned 

In [11]:
print('The dataframe has {} postal codes and {} neighborhoods.'.
     format(df_new.shape[0],len(df_new['Neighborhood'].unique())))

The dataframe has 103 postal codes and 99 neighborhoods.


## Second part

Let's load the values of latitudes and longitudes. 

In [12]:
data_lat=pd.read_csv('Geospatial_Coordinates.csv')

In [13]:
df_final=pd.merge(df_new,data_lat,how='inner')
df_final.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Third part

In [14]:
import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

In [48]:
#df_final['Borough'].unique()

Use geopy library to get the latitude and longitude values of Toronto.

In [15]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Create a map of Toronto with neighborhoods superimposed on top.

In [16]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_final['Latitude'], df_final['Longitude'], df_final['Borough'], df_final['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Folium is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

Llet's simplify the above map and segment and cluster only the neighborhoods in York. So let's slice the original dataframe and create a new dataframe of the York data.

In [17]:
york_data = df_final[df_final['Borough'] == 'York'].reset_index(drop=True)
york_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
1,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
2,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013
3,M6N,York,"Runnymede, The Junction North",43.673185,-79.487262
4,M9N,York,Weston,43.706876,-79.518188


In [18]:
#Let's get the geographical coordinates of York.
address = 'York, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of York are 43.6896191, -79.479188.


In [16]:
#let's visualizat York the neighborhoods in it.
map_york = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(york_data['Latitude'], york_data['Longitude'], york_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_york)  
    
map_york

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.



In [19]:
CLIENT_ID = 'VT5DTAMJ3XFEBH5LMV441H5FYK4DWQ4IJ1RQ2H5RNP1FHLQY' 
CLIENT_SECRET = 'EQEYMPHSHBO1ADOZVKKDHRLN4HFW2P2BKKF1S24AZSTDMNLY' 
VERSION = '20180605' 

In [20]:
york_data.loc[0,'Neighborhood']

'Humewood-Cedarvale'

In [21]:
neighborhood_latitude = york_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = york_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = york_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Humewood-Cedarvale are 43.6937813, -79.42819140000002.


Now, let's get the top 100 venues that are in Humewood-Cedarvale within a radius of 500 meters.


In [22]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


In [23]:
results = requests.get(url).json()
#results

From the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

#nearby_venues.head()

C:\Users\KIIT\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [26]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


Let's create a function to repeat the same process to all the neighborhoods in York

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called york_venues.

In [28]:
york_venues = getNearbyVenues(names=york_data['Neighborhood'],
                                   latitudes=york_data['Latitude'],
                                   longitudes=york_data['Longitude']
                                  )

Humewood-Cedarvale
Caledonia-Fairbanks
Del Ray, Mount Dennis, Keelsdale and Silverthorn
Runnymede, The Junction North
Weston


In [29]:
#Let's check the size of the resulting dataframe
print(york_venues.shape)
york_venues.head()

(19, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Humewood-Cedarvale,43.693781,-79.428191,Cedarvale Park,43.692535,-79.428705,Field
1,Humewood-Cedarvale,43.693781,-79.428191,Cedarvale Ravine,43.690188,-79.426106,Trail
2,Humewood-Cedarvale,43.693781,-79.428191,Glen Cedar Park,43.695399,-79.429253,Playground
3,Humewood-Cedarvale,43.693781,-79.428191,Phil White Arena,43.691303,-79.431761,Hockey Arena
4,Caledonia-Fairbanks,43.689026,-79.453512,Nairn Park,43.690654,-79.456300,Park


In [30]:
#Let's check how many venues were returned for each neighborhood
york_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Caledonia-Fairbanks,4,4,4,4,4,4
"Del Ray, Mount Dennis, Keelsdale and Silverthorn",5,5,5,5,5,5
Humewood-Cedarvale,4,4,4,4,4,4
"Runnymede, The Junction North",4,4,4,4,4,4
Weston,2,2,2,2,2,2


In [31]:
#Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(york_venues['Venue Category'].unique())))

There are 16 uniques categories.


####  Analyze Each Neighborhood

In [32]:
# one hot encoding
york_onehot = pd.get_dummies(york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
york_onehot['Neighborhood'] = york_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [york_onehot.columns[-1]] + list(york_onehot.columns[:-1])
york_onehot = york_onehot[fixed_columns]

york_onehot.head()

,Neighborhood,Bar,Breakfast Spot,Brewery,Bus Line,Convenience Store,Discount Store,Fast Food Restaurant,Field,Grocery Store,Hockey Arena,Park,Playground,Pool,Sandwich Place,Trail,Women's Store
0,Humewood-Cedarvale,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,Humewood-Cedarvale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,Humewood-Cedarvale,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,Humewood-Cedarvale,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,Caledonia-Fairbanks,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [33]:
#And let's examine the new dataframe size.
york_onehot.shape

(19, 17)

In [34]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
york_grouped = york_onehot.groupby('Neighborhood').mean().reset_index()
york_grouped

,Neighborhood,Bar,Breakfast Spot,Brewery,Bus Line,Convenience Store,Discount Store,Fast Food Restaurant,Field,Grocery Store,Hockey Arena,Park,Playground,Pool,Sandwich Place,Trail,Women's Store
0,Caledonia-Fairbanks,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.5,0.00,0.25,0.0,0.00,0.25
1,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",0.2,0.00,0.00,0.00,0.0,0.2,0.4,0.00,0.00,0.00,0.0,0.00,0.00,0.2,0.00,0.00
2,Humewood-Cedarvale,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.25,0.00,0.25,0.0,0.25,0.00,0.0,0.25,0.00
3,"Runnymede, The Junction North",0.0,0.25,0.25,0.25,0.0,0.0,0.0,0.00,0.25,0.00,0.0,0.00,0.00,0.0,0.00,0.00
4,Weston,0.0,0.00,0.00,0.00,0.5,0.0,0.0,0.00,0.00,0.00,0.5,0.00,0.00,0.0,0.00,0.00


In [35]:
#Let's confirm the new size
york_grouped.shape

(5, 17)

In [36]:
#Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in york_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = york_grouped[york_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Caledonia-Fairbanks----
            venue  freq
0            Park  0.50
1            Pool  0.25
2   Women's Store  0.25
3             Bar  0.00
4  Breakfast Spot  0.00


----Del Ray, Mount Dennis, Keelsdale and Silverthorn----
                  venue  freq
0  Fast Food Restaurant   0.4
1                   Bar   0.2
2        Discount Store   0.2
3        Sandwich Place   0.2
4        Breakfast Spot   0.0


----Humewood-Cedarvale----
          venue  freq
0         Field  0.25
1  Hockey Arena  0.25
2    Playground  0.25
3         Trail  0.25
4           Bar  0.00


----Runnymede, The Junction North----
            venue  freq
0  Breakfast Spot  0.25
1         Brewery  0.25
2        Bus Line  0.25
3   Grocery Store  0.25
4             Bar  0.00


----Weston----
               venue  freq
0  Convenience Store   0.5
1               Park   0.5
2                Bar   0.0
3     Breakfast Spot   0.0
4            Brewery   0.0




Let's put that into a pandas dataframe
First, let's write a function to sort the venues in descending order.

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = york_grouped['Neighborhood']

for ind in np.arange(york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(york_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Caledonia-Fairbanks,Park,Women's Store,Pool,Trail,Sandwich Place,Playground,Hockey Arena,Grocery Store,Field,Fast Food Restaurant
1,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",Fast Food Restaurant,Sandwich Place,Discount Store,Bar,Women's Store,Trail,Pool,Playground,Park,Hockey Arena
2,Humewood-Cedarvale,Trail,Playground,Hockey Arena,Field,Women's Store,Sandwich Place,Pool,Park,Grocery Store,Fast Food Restaurant
3,"Runnymede, The Junction North",Grocery Store,Bus Line,Brewery,Breakfast Spot,Women's Store,Trail,Sandwich Place,Pool,Playground,Park
4,Weston,Park,Convenience Store,Women's Store,Trail,Sandwich Place,Pool,Playground,Hockey Arena,Grocery Store,Field


## Cluster Neighborhood

Run k-means to cluster the neighborhood into 3 clusters.

In [39]:
# set number of clusters
kclusters = 5

york_grouped_clustering = york_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 1, 3, 2, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [42]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

york_merged = york_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
york_merged = york_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

york_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M6C,York,Humewood-Cedarvale,43.693781,-79.428191,3,Trail,Playground,Hockey Arena,Field,Women's Store,Sandwich Place,Pool,Park,Grocery Store,Fast Food Restaurant
1,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,4,Park,Women's Store,Pool,Trail,Sandwich Place,Playground,Hockey Arena,Grocery Store,Field,Fast Food Restaurant
2,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013,1,Fast Food Restaurant,Sandwich Place,Discount Store,Bar,Women's Store,Trail,Pool,Playground,Park,Hockey Arena
3,M6N,York,"Runnymede, The Junction North",43.673185,-79.487262,2,Grocery Store,Bus Line,Brewery,Breakfast Spot,Women's Store,Trail,Sandwich Place,Pool,Playground,Park
4,M9N,York,Weston,43.706876,-79.518188,0,Park,Convenience Store,Women's Store,Trail,Sandwich Place,Pool,Playground,Hockey Arena,Grocery Store,Field


Finally, let's visualize the resulting clusters

In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(york_merged['Latitude'], york_merged['Longitude'], york_merged['Neighborhood'], york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [44]:
york_merged.loc[york_merged['Cluster Labels'] == 0, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,York,0,Park,Convenience Store,Women's Store,Trail,Sandwich Place,Pool,Playground,Hockey Arena,Grocery Store,Field


In [45]:
york_merged.loc[york_merged['Cluster Labels'] == 1, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,York,1,Fast Food Restaurant,Sandwich Place,Discount Store,Bar,Women's Store,Trail,Pool,Playground,Park,Hockey Arena


In [46]:
york_merged.loc[york_merged['Cluster Labels'] == 2, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,York,2,Grocery Store,Bus Line,Brewery,Breakfast Spot,Women's Store,Trail,Sandwich Place,Pool,Playground,Park


In [47]:
york_merged.loc[york_merged['Cluster Labels'] == 3, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,York,3,Trail,Playground,Hockey Arena,Field,Women's Store,Sandwich Place,Pool,Park,Grocery Store,Fast Food Restaurant


In [48]:
york_merged.loc[york_merged['Cluster Labels'] == 4, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,York,4,Park,Women's Store,Pool,Trail,Sandwich Place,Playground,Hockey Arena,Grocery Store,Field,Fast Food Restaurant
